In [ ]:
!git clone https://github.com/unfiltered-syrup/TECH-UB.0024_Final

In [ ]:
!pip install flask-ngrok > /dev/null
!pip install -q pyngrok
!ngrok authtoken 23FD4t7XWxBjf9HXFzSOjo7kgbG_5jnc58AtdCLCkr8J6nh3b # <-- replace with your own

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request as flask_request
from flask import Flask, redirect, url_for, render_template
import sys
import json
import webbrowser
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
import nltk
nltk.download('punkt')
import pandas as pd
import re
import string
import spacy
sys.path.append('/content/TECH-UB.0024_Final/TECH-Final-Frontend/py_package')
emotion = ''
with open('/content/TECH-UB.0024_Final/TECH-Final-Frontend/emotion_complete.csv', encoding="ISO-8859-1") as fp:
  cl = NaiveBayesClassifier(fp, format="csv")
string = 'asdfasdf'

app = Flask(__name__, template_folder='/content/TECH-UB.0024_Final/TECH-Final-Frontend/', static_folder='/content/TECH-UB.0024_Final/TECH-Final-Frontend/')

run_with_ngrok(app)
input = 'Im so happy!'
def before_request():
    app.jinja_env.cache = {}
@app.route("/")
def testing():
    return render_template("front_page.html")


@app.route('/get_strings', methods=['POST'])
def get_string():
  global tracks
  alphabets= "([A-Za-z])"
  prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
  suffixes = "(Inc|Ltd|Jr|Sr|Co)"
  starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
  acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
  websites = "[.](com|net|org|io|gov)"

  v2 = pd.read_excel(open('/content/TECH-UB.0024_Final/vent_library.xlsx', 'rb'),sheet_name='vent_library')
  print(v2)
  happy_df = v2[v2['Mood'] == 'Happy']
  angry_df = v2[v2['Mood'] == 'Angry']
  sad_df = v2[v2['Mood'] == 'Sad']
  heartbreak_df = v2[v2['Mood'] == 'Heartbreak']
  focus_df = v2[v2['Mood'] == 'Focus']
  selflove_df = v2[v2['Mood'] == 'Self Love']
  inspiration_df = v2[v2['Mood'] == 'Inspiration']
  party_df = v2[v2['Mood'] == 'Party']
  love_df = v2[v2['Mood'] == 'Love']
  friendship_df = v2[v2['Mood'] == 'Friendship']

  happy_songs = happy_df[['Track']]
  angry_songs = angry_df[['Track']]
  sad_songs = sad_df[['Track']]
  heartbreak_songs = heartbreak_df[['Track']]
  focus_songs = focus_df[['Track']]
  selflove_songs = selflove_df[['Track']]
  inspiration_songs = inspiration_df[['Track']]
  party_songs = party_df[['Track']]
  love_songs = love_df[['Track']]
  friendship_songs = friendship_df[['Track']]

  emotion_list=[]
  text = flask_request.form["text_input"]
  print(text)
  text = " " + text + "  "
  text = text.replace("\n"," ")
  text = re.sub(prefixes,"\\1<prd>",text)
  text = re.sub(websites,"<prd>\\1",text)
  if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
  text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
  text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
  text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
  text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
  text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
  text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
  text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
  if "”" in text: text = text.replace(".”","”.")
  if "\"" in text: text = text.replace(".\"","\".")
  if "!" in text: text = text.replace("!\"","\"!")
  if "?" in text: text = text.replace("?\"","\"?")
  text = text.replace(".",".<stop>")
  text = text.replace("?","?<stop>")
  text = text.replace("!","!<stop>")
  text = text.replace("<prd>",".")
  sentences = text.split("<stop>")
  sentences = sentences[:-1]
  sentences = [s.strip() for s in sentences]
  tracks = []
  for sent in sentences:
    blob=TextBlob(sent,classifier=cl)
    emotion_list.append(blob.classify())
  print(emotion)
  for x in emotion_list:
        if x == 'happiness':
            tracks+=happy_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'anger':
            tracks+=angry_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'sadness':
            tracks+=sad_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'heartbroken':
            tracks+=heartbreak_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'working':
            tracks+=focus_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'worry':
            tracks+=selflove_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'boredom':
            tracks+=inspiration_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'fun':
            tracks+=party_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'love':
            tracks+=love_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'lonely':
            tracks+=friendship_songs.sample(30//len(emotion_list))['Track'].to_list()
        results()
  print(tracks)
  print(type(tracks))
  return render_template("front_page2.html", param1 = json.dumps(tracks))
  
@app.route('/return_result', methods=["GET"])
def results():
  print('result function is called')
  return render_template("front_page2.html", param1 = tracks)

if __name__ == '__main__':
  app.jinja_env.auto_reload = True
  app.config['TEMPLATES_AUTO_RELOAD'] = True
  app.run()

In [ ]:
!mkdir templates/

In [ ]:
!mv TECH-UB.0024_Final/font-end/* templates